In [64]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import io
 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import nltk
from nltk.corpus import stopwords
 
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
 
from google.colab import files
 
import os
from zipfile import ZipFile

In [65]:
#import yelp_util
#Yelp data

yelp_data = "https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/yelp_data.txt"
url = yelp_data


# URLs for dataset- 

Amazon data : https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/amazon_cells_labelled.txt

Yelp data : https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/yelp_data.txt

# Type the name of the dataset in 'url', 
# like this -> url = Yelp data

In [50]:
amazon_data = "https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/amazon_cells_labelled.txt" 

yelp_data = "https://raw.githubusercontent.com/chaitanyaupp18/Sentiment_analysis/main/yelp_data.txt"

url = yelp_data

In [66]:
import pandas as pd

df = pd.read_csv(url, header=None, delimiter="\t")

In [67]:
sen = df[0]
label = df[1]

In [68]:
train_sentences = sen[:950]
train_labels = label[:950]

valid_sentences = sen[950:995]
valid_labels = label[950:995]

test_sentences = sen[995:]
test_labels = label[995:]

In [69]:
print("Length of training sentences: ",len(train_sentences))
print("Length of validation sentences: ",len(valid_sentences))

Length of training sentences:  950
Length of validation sentences:  45


In [70]:
# Tokenizing the sentences and padding

vocab_size = 2000
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 950

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(train_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

valid_sequences = tokenizer.texts_to_sequences(valid_sentences)
valid_padded = pad_sequences(valid_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

training_padded = np.array(training_padded)
train_labels = np.array(train_labels)
valid_padded = np.array(valid_padded)
valid_labels = np.array(valid_labels)

# Modeling Part

Changes that can be made on the model, by using:

1) a different Optimizer (Adam, RMSprop, Adagrad, SGD etc.)

2) a different Learning rate

3) different number of sizes of neural network

4) different activation functions (relu,elu,tanh,sigmoid   etc.)


In [71]:
 #metric = tf.metrics.f1-score(num_classes=2, threshold=0.5)

#ADAM(learning_rate=0.001)

#optim = tf.keras.optimizers.RMSprop(learning_rate=0.001)
#Changing Learning Rate
optim = tf.keras.optimizers.RMSprop(learning_rate=0.001)
#optim = tf.keras.optimizers.Adagrad(learning_rate=0.001)
#optim = tf.keras.optimizers.Adam(learning_rate=0.001)
#optim = tf.keras.optimizers.SGD(learning_rate=0.001)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=True),input_shape=(64,)),
    tf.keras.layers.Dense(50, input_shape=(250,), activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    #tf.keras.len(model.layers),

])

model.summary()

model.compile(loss='binary_crossentropy',optimizer=optim, metrics=['accuracy']) 

num_epochs = 20
history = model.fit(training_padded, train_labels, epochs=num_epochs, 
                    validation_data=(valid_padded, valid_labels), verbose=2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          128000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 400)         424000    
_________________________________________________________________
dense_8 (Dense)              (None, None, 50)          20050     
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
dense_9 (Dense)              (None, None, 1)           51        
Total params: 572,101
Trainable params: 572,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
30/30 - 4s - loss: 0.6938 - accuracy: 0.5092 - val_loss: 0.7185 - val_accuracy: 0.1227
Epoch 2/20
30/30

# Test sentences!

In [72]:
original_test_sentences = []
original_test_labels = []
for i,v in enumerate(test_sentences):
  original_test_sentences.append(v)
for i,v in enumerate(test_labels):
  original_test_labels.append(v)

In [73]:
custom_sentences = ['Nice phone','Very good model','It is working fine','Better architecture','Not so good']
custom_labels = [1,1,1,1,0]

original_test_sentences.extend(custom_sentences)
original_test_labels.extend(custom_labels)

test_sequences = tokenizer.texts_to_sequences(original_test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_padded = np.array(test_padded)
original_test_labels = np.array(original_test_labels)

# Predict the sentiment of test sentences

In [74]:
predicted = model.predict(test_padded)

y_pred = []
for i in range(len(predicted)):
  sum=0
  for j in range(len(predicted[i])):
    sum+=predicted[i][j]
  sum = sum/len(predicted[i])
  if sum>0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

In [75]:
y_pred, original_test_labels

([0, 1, 0, 0, 0, 1, 1, 1, 1, 1], array([0, 0, 0, 0, 0, 1, 1, 1, 1, 0]))

# Scikit-learn Classification Report

In [76]:
print(classification_report(original_test_labels, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         6
           1       0.67      1.00      0.80         4

    accuracy                           0.80        10
   macro avg       0.83      0.83      0.80        10
weighted avg       0.87      0.80      0.80        10



# Model Evaluation

In [77]:
model.evaluate(test_padded,original_test_labels,verbose=2)

1/1 - 0s - loss: 1.4889 - accuracy: 0.8030


[1.488912582397461, 0.8029999732971191]

# Give a name to the model

In [ ]:
model_name = "ImprovedBiLSTM" # Don't give space between the words

model.save('/content/'+model_name)

INFO:tensorflow:Assets written to: /content/ImprovedBiLSTM/assets


INFO:tensorflow:Assets written to: /content/ImprovedBiLSTM/assets


In [ ]:
def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths 

# Save the model and download the model to local drive

In [ ]:
directory = "/content/"+model_name+"/"

file_paths = get_all_file_paths(directory)

with ZipFile('/content/'+model_name+'.zip','w') as zip:
        for file_ in file_paths:
            zip.write(file_)

files.download('/content/'+model_name+'.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>